In [1]:
# Import custom libraries from local folder.
from importlib import reload
import os

# Get current working directory path for the tool parent folder and print it.
from pathlib import Path
import os
parent_folder = 'scapy'
cwd = str(Path(os.getcwd()[:os.getcwd().index(parent_folder)+len(parent_folder)]))

# Import sys library to import libraries from parent folders
import sys
sys.path.append(cwd)

# Import utils library containing miscellaneous functions/classes
from scapy import utils


print(utils.cwd)

/Users/jjrr/Documents/SCA-Project/scapy


In [2]:
import types
def get_attributes(input_module:types.ModuleType, attr_type:str= 'all', 
                   sort:bool=True) -> list:

    attributes = []
    for a in  input_module.__dict__:
        if callable(getattr(input_module, a)) and \
        str(getattr(input_module,a).__module__).startswith('scapy'):
            
            if (attr_type == 'function' and \
                isinstance(getattr(input_module, a), types.FunctionType)) or \
                (attr_type == 'class' and not \
                isinstance(getattr(input_module, a), types.FunctionType)) or \
                attr_type == 'all':
                
                attributes.append(a)

    if sort: attributes = sorted(attributes)

    return attributes

In [3]:
def docstring_to_item(docstring:dict) -> str:

    name = '\t\\item \\pythonfunction{' + docstring['name'] + '}'
    description = '\t\t\\item\\textbf{Description}' + \
                  f":\\\\{docstring['description']}"
    
    # Check if the function returns any output.
    returns = '\t\t\\item\\textbf{Returns}'

    if 'returns' in docstring.keys():
        # Get dictionary from the returns key
        returns = docstring['returns']

        # Convert returns to an items.
        returns = '\t\t\\item\\textbf{Returns}' + \
                f" ({returns['type']}):\\\\{returns['description']}"
    else:
        returns = returns + ': None'

    # Get all the arguments from the function into an itemize list

    if 'args' in docstring.keys():
        args = []
        for arg in docstring['args']:
            args.append(f"\t\t\t\\item {arg['arg_name']} " + \
                        f"({arg['arg_type']}): {arg['arg_description']}")
        args = '\t\t\\item\\textbf{Parameters}:' + \
            '\n\t\t\\begin{itemize}[itemsep=1ex]\n' + \
            '\n'.join(args) + '\n\t\t\\end{itemize}\n'
    else:
        args = '\t\t\\item\\textbf{Parameters}: None'

    # Add begin and end tags for itemize
    itemize = f'{name}\n' + '\t\\begin{itemize}[itemsep=1ex]' + \
              f'\n{description}\n{returns}\n{args}'+ '\t\\end{itemize}'
    return itemize

In [4]:
def get_docstring(method, latex_format:bool=False, return_itemize:bool=False) -> dict:

    if latex_format:
        latex_code = lambda x: '\\texttt{' + str(x) + '}'
    else:
        latex_code = lambda x: str(x)

    name = latex_code(method.__name__ + '()')
    if latex_format: name = name.replace('_','\_')

    docstring = {'name': name}

    reading = 'description'

    for line in method.__doc__.split('\n'):
        if len(line)==0: continue

        if latex_format: line = line.replace('_', '\_')
        
        
        if any(s in line for s in ['Args:', 'Returns:', 'Raises:']):
            reading = line.strip()[:-1].lower()
            continue

        # If section being read is for errors skip the rest of the loop
        if reading == 'raises': continue

        nexo = ' ' if not '):' in line else '\n'

        if reading in docstring.keys(): 
            docstring[reading] = f'{docstring[reading]}{nexo}{line.strip()}'
        else:
            docstring[reading] = line.strip()

    for k in docstring.keys(): 
        if k == 'args': 
            # Split arguments by new lines
            docstring[k] = docstring[k].split('\n')

            # Iterate over all parameters
            for a, arg in enumerate(docstring[k]):

                desc = arg.split('):')[-1].strip()
                name = latex_code(arg.split('):')[0].split('(')[0].strip())
                dtype = latex_code(arg.split('):')[0].split('(')[1].strip())

                docstring[k][a] = {'arg_name':name, 
                                'arg_type': dtype, 
                                'arg_description': desc}
        elif k=='returns':
            dtype = latex_code(docstring[k].split(':')[0])
            desc = docstring[k].split(':')[1]
            docstring[k] = {'type': dtype, 'description': desc}

    if return_itemize:
        return docstring_to_item(docstring)
    else:
        return docstring

In [8]:

from scapy import ccsds, cdm, cef, cells, cre, data, event, layers, sdg, utils


def save_docstring(filepath:str, docstrings:list) -> None:
    with open(filepath, "w") as text_file:
        text_file.write('\\begin{itemize}\n' + \
                        '\n'.join(docstrings) + \
                        '\n\\end{itemize}')

# Get all modules contained in scapy
scapy_modules = ['ccsds', 'cdm', 'cef', 'cells', 'cre', 'data', 'event', 
                 'layers', 'sdg', 'utils']
scapy_modules = [ccsds, cdm, cef, cells, cre, data, event, layers, sdg, utils]

# Iterate over all scapy modules
for module in scapy_modules:
    reload(module)
    
    module_name = module.__name__.split('.')[-1]
    # Print module name
    print(f'MODULE: {module.__name__}')

    # Get all user defined classes and functions
    attributes = get_attributes(input_module = module)

    # Initialise list to store docstrings
    docstrings = []

    # Initialise dictionary to separate docummented functions from 
    # non-documented
    docs = dict(doc = [], und = [])

    # Iterate through all attributes.
    for attr in attributes:
        # Get method using the attribute name
        method = getattr(module, attr)

        if not module_name in str(getattr(module,attr).__module__): continue

        # Check if it is a function or a class.
        if isinstance(method, types.FunctionType):
            # Try to get docstrings from all functions.
            try:
                docstrings.append(get_docstring(method, True, True))
                docs['doc'].append(attr)
            except:
                docs['und'].append(attr)
        else:

            # Initialise dictionary to separate docummented functions from 
            # non-documented
            c_name = attr.lower()
            if not attr[0].isupper(): continue
            c_docs = dict(doc = [], und = [])
            c_docstrings = []

            # If it is a class, get the functions embeded
            c_attributes = get_attributes(method)

            # Iterate over all functions in the class
            for c_attr in c_attributes:
                c_method = getattr(method, c_attr)

                # Try to get all docstrings from functions embeded in the class.
                try:
                    c_docstrings.append(get_docstring(c_method, True, True))
                    c_docs['doc'].append(c_attr)
                except:
                    c_docs['und'].append(c_attr)
                    
            print(f'\t{"Class:":12} (D:{len(c_docs["doc"]):2} | '
                  f'U:{len(c_docs["und"]):2}): {c_name} {c_docs["und"]}')
            
            # Export docstring documentation to external latex file.
            filename = f'docstring-{module_name}-class-{c_name}.tex'.lower()
            filepath = os.path.join(utils.cwd, 'docs', 'tex', filename)
            save_docstring(filepath, c_docstrings)

    if len(docs["doc"])==0: continue
    # Export docstring documentation to external latex file.
    filename = f'docstring-{module_name}-functions.tex'.lower()
    filepath = os.path.join(utils.cwd, 'docs', 'tex', filename)
    save_docstring(filepath, docstrings)

    print(f'\t{"Functions":12} (D:{len(docs["doc"]):2} | '
          f'U:{len(docs["und"]):2}): {docs["und"]}')



MODULE: scapy.ccsds
	Functions    (D: 2 | U: 0): []
MODULE: scapy.cdm
	Class:       (D:27 | U: 0): conjunctiondatamessage []
MODULE: scapy.cef
	Class:       (D:11 | U: 0): conjunctioneventforecaster []
	Class:       (D: 4 | U: 0): cosinewarmupscheduler []
MODULE: scapy.cells
	Class:       (D: 2 | U: 0): bias []
	Class:       (D: 3 | U: 0): gru_mutx []
	Class:       (D: 3 | U: 0): gru_slimx []
	Class:       (D: 3 | U: 0): gru_vanilla []
	Class:       (D: 3 | U: 0): lstm_cifg []
	Class:       (D: 3 | U: 0): lstm_fb1 []
	Class:       (D: 3 | U: 0): lstm_nxg []
	Class:       (D: 3 | U: 0): lstm_nxgaf []
	Class:       (D: 3 | U: 0): lstm_pc []
	Class:       (D: 3 | U: 0): lstm_slimx []
	Class:       (D: 3 | U: 0): lstm_vanilla []
	Class:       (D: 3 | U: 0): mgu_slimx []
	Class:       (D: 3 | U: 0): mgu_vanilla []
MODULE: scapy.cre
	Class:       (D: 7 | U: 0): collisionriskevaluator []
MODULE: scapy.data
	Class:       (D: 5 | U: 0): dataset []
	Class:       (D: 4 | U: 0): tensordatasetfromd

In [ ]:
multirow = lambda x, y: '\\multirow{' + str(x) + '}{*}{' + str(y) + '}' if x>1 else str(y)
rows = len(docstring['arguments'])
latex = ''


latex_table = dict(name=[], parameter=[], parameter_type=[], parameter_description=[])

for a, arg in enumerate(docstring['arguments']):

    if a==0:
        label = '\\texttt{' + info['name'] + '}' + \
                '\n\\textbf{Description}: ' + info['description'] + \
                '\n\\textbf{Returns}:' + info['returns']
        
        label = multirow(rows, label)
    else:
        label=' '
    print(label)
    latex_table['name'].append(label)
    # latex_table['name'].append('\\texttt{' + multirow(rows, info['name']) if a == 0 else ' ')
    # latex_table['description'].append(multirow(rows,info['description']) if a == 0 else ' ')
    # latex_table['returns'].append(multirow(rows,info['returns']) if a == 0 else ' ')
    latex_table['parameter'].append(arg['parameter'])
    latex_table['parameter_type'].append(arg['type'])
    latex_table['parameter_description'].append(arg['description'])
    #latex_table['arguments'].append(' & '.join([arg['parameter'], arg['type'], arg['description']]))

columns = ['name', 'description', 'returns', 'arguments']

import pandas as pd

df_latex = pd.DataFrame.from_dict(latex_table, orient='columns')
display(df_latex)

print(utils.df2latex(df=df_latex,include_header=True) \
      .replace('c} &', 'c}\n &') \
      .replace('_','\_') \
      .replace('\\\\', '\\\\ \n'))

# for r in range(len(latex_table['name'])):

#     column_list = [latex_table[c][r] for c in columns]
#     latex = latex + ' & '.join(column_list) + '\\\\ \n'

# print(utils.format_json(info))
# print(latex)